In [1]:
# libraries

import requests
import re
import pandas as pd
import time
import numpy as np
from bs4 import BeautifulSoup


In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
# price of the apartament

def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [4]:
# lan and lon coordinates

def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat,lon

In [5]:
# number of rooms in the apartament

def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number

In [6]:
# total square of apartament

def getTotalSquare(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Общая площадь:|Площадь комнат', table)[1]
    for item in re.split('\s', str(table)):
        if item!='':
            square = item
            break
    return square

In [7]:
# total live square

def getLiveSquare(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Жилая площадь:|Площадь кухни', table)[1]
    for item in re.split('\s', str(table)):
        if item!='':
            square = item
            break
    if (square == '–'): square='None'
    return square

In [8]:
# kitchen square

def getKitchenSquare(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Площадь кухни:|Санузел', table)[1]
    for item in re.split('\s', str(table)):
        if item!='':
            square = item
            break
    if (square == '–'): square='None'
    return square

In [9]:
# balcony 0/1

def getBalcon(flat_page):
    bal = 0
    bal1 = 0
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Балкон:|Лифт', table)[1]
    if (table.find('нет')>0 or table.find('-')>0 or table.find('отсутс')>0):
        bal = 0
    else:
        if (table.find('лодж')>0 or table.find('балк')>0):
            bal = 1
    return(bal)       


In [10]:
# floor of the apartament

def getFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Этаж:|Тип дома', table)[1]
    if (table.find('/')>0):
        a = table.find('/')-1
        i = 1
        num = 0
        while (table[a-i].isdigit()):
            num = num + 10**(i-1)*int(table[a-i])
            i = i + 1
    else:
        for item in re.split('\s', str(table)):
            if (item.isdigit()>0):
                num = item
                break    
    return(num)

In [11]:
# number of floors in house

def getNumberFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Этаж:|Тип дома', table)[1]
    if (table.find('/')>0):
        a = table.find('/')+1
        i = 1
        num = 0
        while (table[a+i].isdigit()):
            num = num*10**(i-1) + int(table[a+i])
            i = i + 1
    else:
           num = 'None'  
    return(num)

In [12]:
# new building 0/1

def getNew(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Тип дома:|Тип продажи', table)[1]
    if (table.find('перв')>0 or table.find('новостро')>0):
        type = 1
    else:
        if (table.find('втор')>0):
            type = 0
        else:
            type = 'None'
    return(type)

In [13]:
# brick, monolith, reinforced concrete or another 1/0

def getMaterial(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Тип дома:|Тип продажи', table)[1]
    if (table.find('кирп')>0 or table.find('монол')>0 or table.find('жб')>0 or table.find('желез')>0):
        type = 1
    else:
        type = 0
    return(type)

In [14]:
# telephone 0/1

def getTelefone(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    table = re.split('Телефон:|Вид из окна', table)[1]
    num = 'None'
    for item in re.split('\s', str(table)):
        if (item == 'да'):
            num = 1
            break
        else:
            if (item == 'нет'):
                num = 0
                break
    return(num)

In [15]:
# distance to mento in min

def getDistMetro(flat_page):
    dist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    dist = html_stripper(dist)
    if (dist!='None'):
        for item in re.split('\s', str(dist)):
                if item!='':
                    dist = item
                    break
                else:    
                    dist = 'None'
    if dist == 'None': 
        dist1 = flat_page.find('div', attrs={'class':'object_descr_text'})
        dist1 = html_stripper(dist1)
        a = dist1.find('мин') - 1
        num = 0
        for i in range(1,3):
            if (dist1[a-i]!=' '):
                if (dist1[a-i].isdigit()):
                    num = num + int(dist1[a-i])*(10**(i-1))
                else: break
            else:
                 break
            if (num>0):
                dist = num
            else: 
                dist = 'None'                
    return dist

In [16]:
# on foot/transport to metro

def getWalkMetro(flat_page):    
    dist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    dist = html_stripper(dist)
    if (dist!='None'):
        if ((dist.find('тран')>0) or (dist.find('маш')>0) or (dist.find('авт')>0) or (dist.find('трам')>0))  :
            ans = 0
        else: 
            if (dist.find('пеш')>0) :
                ans = 1
            else: 
                ans = 'None'
    if dist == 'None': 
        dist1 = flat_page.find('div', attrs={'class':'object_descr_text'})
        dist1 = html_stripper(dist1)
        if ((dist1.find('тран')>0) or (dist1.find('маш')>0) or (dist1.find('авт')>0) or (dist1.find('трам')>0))  :
            ans = 0
        else: 
            if (dist1.find('пеш')>0) :
                ans = 1
            else: 
                ans = 'None'              
    return ans

In [17]:
# Distance to the center of Moscow (Piphagor)

def getDist(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    
    lat1 =  55.755833
    lon1 = 37.617778
    
    delta_lat = (lat - lat1)**2
    delta_lng = (lon - lon1)**2
    
    dist = np.sqrt(delta_lat + delta_lng)
    return dist

In [18]:
# main districts of Moscow and the suburbs

kol = 0

district1 =[]

district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
district1.append('http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1')
Cian = []  

In [19]:
for raon in range(0,10): # all districts
    
    print(raon)
    district = district1[raon]

    links = []

    for page in range(1, 30): # pages of each distroct with restrictions equal to 30

        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls: # all links to apartaments in district
            if link.isdigit():
                links.append(link)   

              
    for i in range(0, len(links)):

        kol = kol+1
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')

        flatStats = {'District':0}

        flatStats['URL'] = str(links[i])
        flatStats['N'] = kol
        flatStats['Price']= getPrice(flat_page)
        flatStats['lat'], flatStats['lon'] = getCoords(flat_page)
        flatStats['rooms'] = getRoom(flat_page)
        flatStats['Totsp'] = getTotalSquare(flat_page)
        flatStats['Livesp'] = getLiveSquare(flat_page)
        flatStats['Kitsp'] = getKitchenSquare(flat_page)
        flatStats['MetrDist'] = getDistMetro(flat_page)
        flatStats['Walk'] = getWalkMetro(flat_page)
        flatStats['Dist'] = getDist(flat_page) 
        flatStats['Bal'] = getBalcon(flat_page)
        flatStats['Floor'] = getFloor(flat_page)
        flatStats['NFloor'] = getNumberFloor(flat_page)
        flatStats['New'] = getNew(flat_page)
        flatStats['Brick'] = getMaterial(flat_page)
        flatStats['Tel'] = getTelefone(flat_page)

        Cian.append(flatStats) # data base
    

0
812
1
812
2
812
3
812
4
812
5
812
6
812
7
812
8
812
9
812


In [20]:
P = pd.DataFrame(Cian)
P.to_csv('cianchik.csv')